In [1]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as nplin
import itertools

In [2]:
np.random.seed(0)

In [3]:
def operators(s):
    #generate terms in the energy function
    n_seq,n_var = s.shape
    ops = np.zeros((n_seq,n_var+int(n_var*(n_var-1)/2.0)))

    jindex = 0
    for index in range(n_var):
        ops[:,jindex] = s[:,index]
        jindex +=1

    for index in range(n_var-1):
        for index1 in range(index+1,n_var):
            ops[:,jindex] = s[:,index]*s[:,index1]
            jindex +=1
            
    return ops

In [4]:
def energy_ops(ops,w):
    return np.sum(ops*w[np.newaxis,:],axis=1)

In [5]:
def generate_seqs(n_var,n_seq,n_sample=30,g=1.0):
    samples = np.random.choice([1.0,-1.0],size=(n_seq*n_sample,n_var),replace=True)
    ops = operators(samples)
    n_ops = ops.shape[1]

    #w_true = g*(np.random.rand(ops.shape[1])-0.5)/np.sqrt(float(n_var))
    w_true = np.random.normal(0.,g/np.sqrt(n_var),size=n_ops)

    sample_energy = energy_ops(ops,w_true)
    p = np.exp(sample_energy)
    p /= np.sum(p)
    out_samples = np.random.choice(np.arange(n_seq*n_sample),size=n_seq,replace=True,p=p)
    
    return w_true,samples[out_samples] #,p[out_samples],sample_energy[out_samples]

In [6]:
def eps_machine(s,eps_scale=.01,max_iter=151,alpha=0.1):
    MSE = np.zeros(max_iter)
    KL = np.zeros(max_iter)
    E_av = np.zeros(max_iter)
    
    n_seq,n_var = s.shape
    ops = operators(s)
    n_ops = ops.shape[1]
    cov_inv = np.eye(ops.shape[1])

    np.random.seed(13)
    w = np.random.rand(n_ops)-0.5
    for i in range(max_iter): 
        
        eps_scale = np.random.rand()/np.max([1.,np.max(np.abs(w))])
        
        energies_w = energy_ops(ops,w)
        probs_w = np.exp(-energies_w*(1-eps_scale))
        z_data = np.sum(probs_w)
        probs_w /= z_data
        ops_expect_w = np.sum(probs_w[:,np.newaxis]*ops,axis=0)
        
        #if iterate%int(max_iter/5.0)==0:
        E_exp = (probs_w*energies_w).sum()
        KL[i] = -E_exp - np.log(z_data) + np.sum(np.log(np.cosh(w*eps_scale))) + n_var*np.log(2.)
        
        E_av[i] = energies_w.mean()
        MSE[i] = ((w-w_true)**2).mean()
        #print(RMSE[i])
        
        #print(eps_scale,iterate,nplin.norm(w-w_true),RMSE,KL,E_av)

        sec_order = w*eps_scale
        w += alpha*cov_inv.dot((ops_expect_w - sec_order))
        
    #print('final ',eps_scale,iterate,nplin.norm(w-w_true))
    
    return MSE,KL,-E_av

In [7]:
max_iter = 100

n_var,n_seq = 40,5000
g = 1.

w_true,seqs = generate_seqs(n_var,n_seq,g=g)

#eps_list = np.linspace(0,1.0,11)
eps_list = [0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.99]
n_eps = len(eps_list)

#w = np.zeros((n_eps,max_iter,))
#MSE = np.zeros((n_eps,max_iter))
#KL = np.zeros((n_eps,max_iter))
#E_av = np.zeros((n_eps,max_iter))

#for i,eps in enumerate(eps_list):     
#    print('eps:',eps)
MSE,KL,E_av = eps_machine(seqs,eps_scale=0.1,max_iter=max_iter)

In [9]:
#np.savetxt('KL_n5k.txt',KL,fmt='%f')
np.savetxt('MSE_random_n5k.txt',MSE,fmt='%f')
np.savetxt('E_av_random_n5k.txt',E_av,fmt='%f')